In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial import distance_matrix
import random
import math
from operator import attrgetter
from ordered_set import OrderedSet
# if you use conda run this in your terminal: conda install -c conda-forge ordered-set
import gurobipy as gb

In [2]:
# local files
from customer import Customer
from locker_customer import LockerCustomer
from door_to_door_customer import DoorToDoorCustomer
from location import Location
from store import *
from vehicle import Vehicle
from vehicle_type import VehicleType
from executor_multi_period import solve_period

In [3]:
# input
custom_setup = False
C = 150
L = 2
periods = 5

map_size = 100
ratio_locker_customers = 0.3

In [4]:
# define gamma and current_day
gamma = []
gamma.append(0.5)

for day in range(1, periods-1):
    gamma.append((50 + (100-50)/(periods-day))/100)
gamma.append(1)
print("gamma: ", gamma)
current_day = 0

C_per_period = []
for p in range(periods-1):
    C_per_period.append(C // periods)

C_per_period.append(C-sum(C_per_period))

print("C_per_period: ", C_per_period)


gamma:  [0.5, 0.625, 0.6666666666666667, 0.75, 1]
C_per_period:  [30, 30, 30, 30, 30]


In [5]:
def get_nearest_store(stores, location):
    min_distance = float("inf")
    nearest_store = stores[0]
    for store in stores:
        distance = location.euclidean_distance(store.location)
        if distance < min_distance:
            min_distance = distance
            nearest_store = store
    return nearest_store

In [6]:
def generate_C_customers(C, stores):
    customers = []
    counter_locker_customer = 0
    counter_door_to_door_customer = 0
    for c in range(C):
        location = Location(random.randint(0, map_size), random.randint(0, map_size))
        if random.random() < ratio_locker_customers:
            # customer locker
            customers.append(LockerCustomer(c, counter_locker_customer, location, get_nearest_store(stores, location)))
            counter_locker_customer += 1
        else:
            # door to door customer
            customers.append(DoorToDoorCustomer(c, counter_door_to_door_customer, location))
            counter_door_to_door_customer += 1
    return customers

In [7]:
# generates stores
if custom_setup:
    L = 2
    stores = []
    stores.append(Store(0, Location(60, 50), capacity=float("inf"), is_warehouse=True))
    stores.append(Store(1, Location(30, 50), capacity=math.ceil(0.8 * C / L)))
    stores.append(Store(2, Location(50, 20), capacity=math.ceil(0.8 * C / L)))

else:
    stores = []
    stores.append(Store(0, Location(random.randint(0, map_size), random.randint(0, map_size)), capacity=float("inf"), is_warehouse=True))
    for l in range(L):
        stores.append(Store(l+1, Location(random.randint(0, map_size), random.randint(0, map_size)), capacity=math.ceil(0.8 * C / L)))

In [8]:
# generate all_customer for all the periods and put them in the right period to be served
all_customers = generate_C_customers(C, stores)
customers_per_period = []
counter_customer = 0
for day in range(periods):
    customers_per_period.append(all_customers[0+counter_customer:counter_customer+C_per_period[day]])
    counter_customer += C_per_period[day]


In [9]:
customers = customers_per_period[current_day]
# create sets
C_L = list(filter(lambda customer: type(customer) == LockerCustomer, customers))
C_D = list(filter(lambda customer: type(customer) == DoorToDoorCustomer, customers))
lockers = list(filter(lambda store: not store.is_warehouse, stores))

# define all the vehicles
sum_W_l = 0
for store in stores:
    if not store.is_warehouse:
        sum_W_l += store.capacity

# define all the vehicles
if custom_setup:
    vehicles = []
    vehicles.append(Vehicle(0, VehicleType.LOCKER_SUPPLY, stores[0], math.ceil(0.8 * sum_W_l)))
    vehicles.append(Vehicle(1, VehicleType.PF, stores[0], math.ceil(0.5 * len(C_D))))
    vehicles.append(Vehicle(2, VehicleType.LF, stores[1], math.ceil(0.6 * stores[1].capacity)))
    vehicles.append(Vehicle(3, VehicleType.LF, stores[2], math.ceil(0.6 * stores[2].capacity)))
else:
    vehicles = []
    for store in stores:
        if store.is_warehouse:
            vehicles.append(Vehicle(0, VehicleType.LOCKER_SUPPLY, store, math.ceil(0.8 * sum_W_l)))
            vehicles.append(Vehicle(1, VehicleType.PF, store, math.ceil(0.5 * len(C_D))))
        else:
            vehicles.append(Vehicle(store.index+1, VehicleType.LF, store, math.ceil(0.6 * store.capacity)))

In [10]:
for day in range(periods):
    print("DAY: ", day)
    if day == 0:
        customers = customers_per_period[current_day]
    C_L = list(filter(lambda customer: type(customer) == LockerCustomer, customers))
    C_D = list(filter(lambda customer: type(customer) == DoorToDoorCustomer, customers))

    status, Runtime, ObjVal, OC, w_c_k_variables = solve_period(stores, vehicles, customers_per_period[current_day])

    print("-------w_c_k_variables-----------")
    print(w_c_k_variables)

    print("----OC-------")
    print(OC)
    if status != 2:
        raise Exception("Gurobi did not find the optimal solution for the model")

    print("day: ", day, "status: ", status)

    # discover the customers that did not get the package
    customer_next_period = []
    customers_did_not_get_the_package = []
    for oc in OC:
        if random.random() > gamma[current_day]:
            customers_did_not_get_the_package.append(oc)
            if oc in w_c_k_variables:
                customers_did_not_get_the_package.append(w_c_k_variables[oc])

    # make all the CD of the previous period prime
    for c in customers_did_not_get_the_package:
        if type(c) == DoorToDoorCustomer:
            c.set_prime(True)

    # add the new customer of the next period
    print("---------customers_did_not_get_the_package----------")
    print(customers_did_not_get_the_package)

    if current_day+1 < periods:
        customer_next_period =  customers_per_period[current_day+1]
        customers = customers_did_not_get_the_package + customer_next_period

    print("--------customer_next_period-----------")
    print(customer_next_period)
    print("--------customers-----------")
    print(customers)

    current_day +=1
    print()

DAY:  0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-11
-------w_c_k_variables-----------
{CL1: CD19, CL2: CD4, CL4: CD17, CL8: CD20}
----OC-------
[CL0, CL1, CL2, CL3, CL4, CL7, CL8]
day:  0 status:  2
---------customers_did_not_get_the_package----------
[CL0, CL1, CD19, CL3]
--------customer_next_period-----------
[CL9, CD21, CL10, CD22, CD23, CD24, CL11, CD25, CD26, CD27, CD28, CL12, CD29, CL13, CD30, CL14, CD31, CD32, CD33, CL15, CD34, CL16, CD35, CD36, CD37, CD38, CD39, CD40, CD41, CD42]
--------customers-----------
[CL0, CL1, CD19, CL3, CL9, CD21, CL10, CD22, CD23, CD24, CL11, CD25, CD26, CD27, CD28, CL12, CD29, CL13, CD30, CL14, CD31, CD32, CD33, CL15, CD34, CL16, CD35, CD36, CD37, CD38, CD39, CD40, CD41, CD42]

DAY:  1
-------w_c_k_variables-----------
{CL12: CD37, CL13: CD29, CL14: CD41, CL16: CD30}
----OC-------
[CL10, CL11, CL12, CL13, CL14, CL16]
day:  1 status:  2
---------customers_did_not_get_the_package----------
[CL10, CL12, C

In [30]:
status, Runtime, ObjVal, OC, w_c_k_variables = solve_period(stores, vehicles, customers_per_period[current_day])

In [31]:
if status != 2:
    raise Exception("Gurobi did not find the optimal solution for the model")

print(OC)
print(w_c_k_variables)

[CL0]
{CL0: CD2}


In [32]:
# discover the customers that did not show up
customer_next_period = []
customers_did_not_get_the_package = []
for oc in OC:
    if random.random() > gamma[current_day]:
        customers_did_not_get_the_package.append(oc)
        if oc in w_c_k_variables:
            customers_did_not_get_the_package.append(w_c_k_variables[oc])

for c in customers_did_not_get_the_package:
    if type(c) == DoorToDoorCustomer:
        c.set_prime(True)

# add the new customer of the next period

print(customers_did_not_get_the_package)


[CL0, CD2]


In [33]:
customer_next_period =  customers_per_period[current_day+1]
print(customer_next_period)
customers = customers_did_not_get_the_package + customer_next_period
print(customers)

[CD7, CD8, CD9, CL3, CD10, CD11, CL4, CD12, CD13, CL5]
[CL0, CD2, CD7, CD8, CD9, CL3, CD10, CD11, CL4, CD12, CD13, CL5]
